In [ ]:
# import modules and define functions
from pyscf import gto, scf
from matplotlib import pyplot as plt, ticker
import matplotlib as mpl
import numpy as np
import os
os.environ['OMP_NUM_THREADS'] = "1" 

def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

In [ ]:
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74'
basis = 'sto-3g'
charge = 0
spin = 0
H2 = gto.Mole()
H2.build(atom=atom,basis=basis,charge=charge,spin=spin)

In [ ]:
# initiate RHF
mf = scf.UHF(H2)
es = []
# run RHF SCF for 1001 c1 mo coefficients
c1s = np.linspace(0,1,101)
c2s = []
mf.max_cycle = 0
s = mf.get_ovlp(H2)
for c1 in c1s:
    c2 = (1+(c1**2)*(s[0][1]**2-1)) ** 0.5 - c1*s[0][1]
    c2s.append(c2)
    mo_coeff = np.array([[c1, c1], [c2, -c2]])
    mo_occ = np.array([2, 0])
    dm1 = make_rdm1(mo_coeff, mo_occ)
    conv, e, mo_e, mo, mo_occ = scf.hf.kernel(mf, dm0=dm1)
    es.append(e)
        

In [ ]:
# plot the change of energy 
xlabel = 'c1**2+c1*c2*S12'
ylabel = 'Energy / En'
title = 'SCF energy vs mo coefficient'
mpl.rcParams['axes.linewidth'] = 3
mpl.rcParams['xtick.major.size'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['xtick.minor.size'] = 5
mpl.rcParams['xtick.minor.width'] = 3
mpl.rcParams['ytick.minor.size'] = 5
mpl.rcParams['ytick.minor.width'] = 3
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
fig, ax = plt.subplots(figsize=(7,6))
ax.xaxis.set_major_locator(ticker.MultipleLocator(0.25))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(0.125))
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))
ax.set_ylim(-1.22,-0.6)
ax.set_xlim(0,1)
ax.set_xlabel(xlabel, fontsize=16)
ax.set_ylabel(ylabel, fontsize=16)
ax.set_title(title, fontsize=22)
ax.scatter(c1s**2+c1s*c2s*s[0][1], es, color='k')